In [3]:
import os
import imp
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import sys
import pickle
import importlib.util
import sympy as sp
from grp import struct_group
sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
import lumapi
sys.path.append(os.path.dirname('/home/moises/Downloads/'))
#os.add_dll_directory('C:\\Program Files\\Lumerical\\v202\\api\\python\\')
#lumapi = imp.load_source('lumapi','C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py') 

In [4]:
fdtd =lumapi.FDTD(filename= 'halfring_bend.fsp')

In [9]:
fdtd.switchtolayout()
fdtd.selectall()
fdtd.delete()

radius = 6e-6
gap = 0.15e-6
Lc = 0e-6
len = 20e-6

fdtd.addobject("ring_resonator")
fdtd.setnamed("ring resonator::outer_bottom",'enabled',0)
fdtd.setnamed("ring resonator::inner_bottom",'enabled',0)
fdtd.setnamed("ring resonator::arc2",'enabled',0)
fdtd.setnamed("ring resonator::arc3",'enabled',0)

fdtd.select("ring resonator")
fdtd.set({"x":0,
"y":0,
"z":0,
"Lc":Lc,
"gap":gap,
"radius":radius,
"base width":0.4e-6,
"base height":0.18e-6,
"x span":len,
"material":"Si (Silicon) - Palik Copy 1"})

In [10]:
fdtd.switchtolayout()
fdtd.select("rect_right")
fdtd.delete()
fdtd.select("rect_left")
fdtd.delete()

x_right_rect = (len/2 + Lc/2 + radius)
x_right_rect_span = 0.4e-6
y_right_rect_span = 2e-6
y_right_rect = -y_right_rect_span/2

x_left_rect = len/2 - Lc/2 - radius
x_left_rect_span = 0.4e-6
y_left_rect_span = 2e-6
y_left_rect = -y_left_rect_span/2

fdtd.addrect()
fdtd.set({"x":x_right_rect,
"x span":x_right_rect_span,
"y":y_right_rect,
"y span":y_right_rect_span,
"z":0,
"z span":0.18e-6,
"material":"Si (Silicon) - Palik Copy 1",
"name":"rect_right"})

fdtd.addrect()
fdtd.set({"x":x_left_rect,
"x span":x_left_rect_span,
"y":y_left_rect,
"y span":y_left_rect_span,
"z":0,
"z span":0.18e-6,
"material":"Si (Silicon) - Palik Copy 1",
"name":"rect_left"})


In [11]:
fdtd.switchtolayout()
fdtd.select("FDTD")
fdtd.delete()

fdtd.addfdtd()
fdtd.set({"x":len/2,
"x span":len - 8.5e-6,
"y min":y_right_rect - 0.25e-6,
"y max":6e-6,
"z":0,
"z span":2e-6,
"z min bc":"Symmetric",
"background material":"SiO2 (Glass) - Palik Copy 1",
"mesh accuracy":4,
"dt stability factor":0.95,
"simulation time":2200e-15})

fdtd.addport()
fdtd.set({"x":len/2 - Lc/2 - radius - 1e-6,
"y":radius + gap + 0.4e-6,
"y span":4e-6,
"z":0,
"z span":2e-6})

fdtd.addport()
fdtd.set({"x":len/2 + Lc/2 + radius + 1e-6,
"y":radius + gap + 0.4e-6,
"y span":4e-6,
"z":0,
"z span":2e-6,
"direction":"Backward"})

fdtd.addport()
fdtd.set({"x":x_left_rect,
"y":y_left_rect,
"x span":4e-6,
"z":0,
"z span":2e-6,
"injection axis":"y-axis",
"direction":"Forward"})

fdtd.addport()
fdtd.set({"x":x_right_rect,
"y":y_right_rect,
"x span":4e-6,
"z":0,
"z span":2e-6,
"injection axis":"y-axis",
"direction":"Forward"})

fdtd.select("FDTD::ports")
fdtd.set({"monitor frequency points":25})
fdtd.setglobalsource("wavelength start",1.5e-6)
fdtd.setglobalsource("wavelength stop", 1.6e-6)


1.0

In [16]:
fdtd.switchtolayout()
fdtd.setnamed("ring resonator::outer_top",'enabled',0)
fdtd.select("ring_up")
fdtd.delete()
fdtd.select("rect_left_up")
fdtd.delete()
fdtd.select("rect_right_up")
fdtd.delete()

radius_up = 6e-6

fdtd.addring()
fdtd.set({"name":"ring_up",
"x":len/2,
"y":radius + gap + 0.4e-6 + radius_up,
"z":0,
"z span":0.18e-6,
"outer radius":radius_up + 0.2e-6,
"inner radius":radius_up - 0.2e-6,
"theta start":180,
"theta stop":0,
"material":"Si (Silicon) - Palik Copy 1"})

fdtd.addrect()
fdtd.set({"x":x_left_rect,
"x span":x_left_rect_span,
"y":radius + gap + 0.4e-6 + radius_up + y_left_rect_span/2,
"y span":y_left_rect_span,
"z":0,
"z span":0.18e-6,
"material":"Si (Silicon) - Palik Copy 1",
"name":"rect_left_up"})

fdtd.addrect()
fdtd.set({"x":x_right_rect,
"x span":x_left_rect_span,
"y":radius + gap + 0.4e-6 + radius_up + y_left_rect_span/2,
"y span":y_left_rect_span,
"z":0,
"z span":0.18e-6,
"material":"Si (Silicon) - Palik Copy 1",
"name":"rect_right_up"})

fdtd.select("FDTD::ports::port 1")
fdtd.set({"injection axis":"y-axis",
"direction":"Backward",
"y":radius + gap + 0.4e-6 + radius_up + y_left_rect_span/2,
"x":x_left_rect,
"x span":4e-6})

fdtd.select("FDTD::ports::port 2")
fdtd.set({"injection axis":"y-axis",
"direction":"Backward",
"y":radius + gap + 0.4e-6 + radius_up + y_left_rect_span/2,
"x":x_right_rect,
"x span":4e-6})

fdtd.setnamed("FDTD","y max", 14e-6)
fdtd.setnamed("FDTD","y min", -1.5e-6)
fdtd.setnamed("FDTD","x span", 18e-6)